디스크립터는 다른 언어에서도 생소한 개념이어서 마땅한 비교대상이 없다.

디스크립터는 파이썬의 객체지향 수준을 한 단계 더 끌어올려주는 혁신적인 기능으로 이 기능을 잘 활용하면 보다 견고하고 재사용성이 높은 추상화를 할 수 있다. 디스크립터의 기능을 제대로 활용하는 예는 라이브러리나 프레임워크에서 많이 발견할 수 있다.

* 디스크립터가 무엇인지 어떻게 동작하는지 어떻게 효율적으로 구현하는지 이해한다.
* 두 가지 유형의 디스크립터 - 데이터 디스크립터와 비데이터 디스크립터의 개념적 차이와 세부구현의 차이를 분석한다.
* 디스크립터를 화용한 코드 재사용 방법
* 디스크립터의 좋은 사용 예를 살펴보고 자체 라이브러리의 API에 어떻게 활용할 수 있는지 살펴본다.

## 디스크립터 개요

먼저 디스크립터에 대한 배경지식을 쌓은 다음 보다 깔끔하고 파이썬스러운 구현을 한 예제를 살펴본다.

### 디스크립터 메커니즘

디스크립터의 동작방식은 그리 복잡하지 않지만, 세부 구현 시의 주의사항이 많다는 점에 유의해야 한다.

디스크립터를 구현하려면 최소 두 개의 클래스가 필요하다. 클라이언트 클래스는 디스크립터 구현의 기능을 활용할 도메인 모델로서 솔루션을 위해 생성한 일반적인 추상화 객체이다. 디스크립터 클래스는 디스크립터 로직의 구현체이다. 

디스크립터는 단지 디스크립터 프로토콜을 구현한 클래스의 인스턴스이다.이 클래스는 다음 매직 메서드 중에 최소 한 개 이상을 포함해야 한다.

* \__get__
* \__set__
* \__delete__
* \__set__name__

다음과 같은 네이밍 컨벤션을 사용한다.

* ClientClass - 디스크립터 구현체의 기능을 활용할 도메인 추상화 객체. 디스크립터의 클라이언트이다. 클래스 속성(인스턴스 속성과 달리 여러 객체가 값을 공유함)으로 디스크립터를 갖는다. 디스크립터는 DescriptorClass의 인스턴스이다.

* DescriptorClass - 디스크립터 클래스. 이 클래스는 앞으로 언급할 디스크립터 프로토콜을 따르는 매직 메서드를 구현해야만 한다.

* client - ClientClass의 인스턴스

* descriptor - DescriptorClass의 인스턴스

명심해야 할 중요한 사실을 이 프로토콜이 동작하려면 디스크립터 객체가 클래스 속성으로 정의되어야 한다는 것이다. 이 객체를 인스턴스 속성으로 생성하면 동작하지 않으므로 init 메서드가 아니라 클래스 본문에 있어야 한다.

디스크립터 프로토콜의 일부만 구현해도 된다는 것에 유의하자. 

디스크립터 클래스와 디스크립터의 로직을 사용하는 크랠스, 즉 디스크립터 객체를 멤버로 갖는 클래스, 이렇게 두 개의 클래스가 있다. ClientClass의 인스턴스에서 descriptor 속성을 호출하면 디스크립터 프로토콜이 사용된다. 이 모든 동작은 러타임 중에 어떻게 작동할까?

다음은 일반적인 클래스의 속성 또는 프로퍼티에 접근하면 예상한 것과 같은 결과를 얻을 수 있다.

In [1]:
class Attribute:
    value = 42

class Client:
    attribute = Attribute()

Client.attribute

In [2]:
Client.attribute.value

42

그러나 디스크립터의 경우 약간 다르게 동작한다. 클래스 속성을 객체로 선언하면 디스크립터로 인식되고, 클라이언트에서 해당 속성을 호출하면 객체 자체를 반환하는 것이 아니라 \__get__ 매직 메서드의 결과를 반환한다.

호출당시의 문맥 정보를 로깅하고 클라이언트 인스턴스를 그대로 반환하는 간단한 예제를 살펴보자.

In [13]:
import logging
import time
import functools
logger = logging.getLogger(__name__)

class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        logger.info("Call: %s.__get__(%r, %r)" %(self.__class__.__name__, instance, owner))
        print("Call: %s.__get__(%r, %r)" %(self.__class__.__name__, instance, owner))

        return instance

class ClientClass:
        descriptor = DescriptorClass()

이제 ClientClass 인스턴스의 descriptor 속성에 접근해보면 DescriptorClass 인스턴스를 반환하지 않고 대신에 \__get__() 메서드의 반환 값을 사용한다는 것을 알 수 있다.

In [14]:
client = ClientClass()
client.descriptor

Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x06CCF930>, <class '__main__.ClientClass'>)


In [15]:
client.descriptor is client

Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x06CCF930>, <class '__main__.ClientClass'>)


True

\__get__ 메서드에 있던 로그가 어떻게 찍혔는지 살펴보자. 이 예제에서는 클라이언트 자체를 그대로 반환했으므로 마지막 비교 문장은 True가 된다. \__get__ 메서드의 파라미턴는 이후 섹션에서 각 메서드별로 자세히 살펴본다. 

간단한 설명용 예제에서 출발하여 더 복잡한 추상화와 더 나은 데코레이터를 만들어 볼 것이다. 디스크립터를 사용하면 완전히 새롭게 프로그램의 제어 흐름을 변경할 수 있다. 이 도구를 사용해 \__get__ 메서드 뒤쪽으로 모든 종류의 논리를 추상화할 수 있으며 클라이언트에게 내용을 숨긴 채로 모든 유형의 변환을 투명하게 실행할 수 있다. 

### 디스크립터 프로토콜의 메서드 탐색

디스크립터의 내부 동작 원리에 대해 알아보자. 디스크립터는 단지 객체이기 때문에 이러한 메서드들은 self를 첫 번째 파라미터로 사용한다. self는 디스크립터 객체 자신을 의미한다. 

디스크립터 프로토콜의 각 메서드에 사용되는 파라미터와 사용 방법에 대해서 알아보자.

### \__get__(self, instance, owner)

두 번째 파라미터 instance는 디스크립터를 호출한 객체를 의미한다. 앞선 예제에서는 client 객체를 의미한다.

owner 파라미터는 해당 객체의 클래스를 의미한다. 이어질 예제에서는 ClientClass로 앞엣 살펴보았던 클래스이다.

앞에서 \__get__ 메서드 시그니처에 있는 instance 파라미터는 디스크립터가 행동을 취하려는 객체이고 owner는 인스턴스의 클래스이다. owner = instance.\__class__와 같은 형태로 instance에서 클래스를 직접 구할 수 있음에도 굳이 owner를 시그니처에 사용하는 이유는 client 인스턴스가 아니라 ClientClass에서 descriptor를 호출하는 특별한 경우로서 instance의 값은 None이기 때문에 클래스를 구할 수 없고, 따라서 굳이 파라미터를 추가하여 owner를 받은 것이다.

간단한 코드를 사용하여 디스크립터가 클래스에서 호출될 때와 인스턴스에서 호출될 때의 차이를 알아보자. \__get__ 메서드는 각각의 경우에 대해 두 개의 개별적인 작업을 수행한다.

In [37]:
class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return f"{self.__class__.__name__}.{owner.__name__}"
        return f"value for {instance}"

class ClientClass:
    descriptor = DescriptorClass()

In [38]:
ClientClass.descriptor

'DescriptorClass.ClientClass'

In [39]:
ClientClass().descriptor

'value for <__main__.ClientClass object at 0x06C3EA50>'

일반적으로 정말 owner 파라미터를 사용하려는 경우가 아니라면, 인스턴스가 None일 때는 단순히 디스크립터 자체를 반환한다.

### \__set__(self, instance, value)

이 메서드는 디스크립터에 값을 할당하려고 할 때 호춛된다. 다음과 같이 \__set__() 메서드를 구현한 디스크립터에 대해서만 활성화된다. 다음 예제의 경우 instance 파라미터는 client이고 value는 "value"라는 문자열이다.

In [40]:
client.descriptor = "value"

In [41]:
client.descriptor

'value'

client.descriptorsms \__set__() 메서드를 구현하지 않았다. 따라서 descriptor 자체를 덮어쓴다.

다음은 이 메서드를 활용하여 속성의 유효성을 검사하는 객체를 어떻게 만들 수 있는지 보여준다. 유효성 검사 함수는 자유롭게 생성할 수 있으며 객체에 값을 할당하기 전에 실행된다. 

In [44]:
class Validation:
    def __init__(self, validation_function, error_msg: str):
        self.validation_function = validation_function
        self.error_msg = error_msg
    
    def __call__(self, value):
        if not self.validation_function(value):
            raise ValueError(f"{value!r} {self.error_msg}")

class Field:
    def __init__(self, *validations):
        self._name = None
        self.validations = validations
    
    def __set_name__(self, owner, name):
        self._name = name
    
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self._name]
    
    def validate(self, value):
        for validation in self.validations:
            validation(value)
    
    def __set__(self, instance, value):
        self.validate(value)
        instance.__dict__[self._name] = value

class ClientClass:
    descriptor = Field(
        Validation(lambda x : isinstance(x, (int, float)), "는 숫자가 아님"),
        Validation(lambda x : x >= 0, "는 0보다 작음"),
    )

위 코드를 실행 하면 다음과 같은 결과가 나온다.

In [45]:
client = ClientClass()
client.descriptor = 42

In [46]:
client.descriptor

42

In [47]:
client.descriptor = -42

ValueError: -42 는 0보다 작음

In [48]:
client.descriptor = 'invalid value'

ValueError: 'invalid value' 는 숫자가 아님

여기서 기억할 점은 프로퍼티 자리에 놓일 수 있는 것은 디스크립터로 추상화할 수 있으며 여러 번 재사용할 수 있다는 것이다. 이 예에서는 \__set__() 메서드가 @property.setter가 하던 일을 대신한다.

### \__delete__(self, instance)

이 메서드는 다음과 같은 형태로 호출된다. self는 descriptor 속성을 나타낸고 instance는 client를 나타낸다.

In [50]:
del client.descriptor

다음 예제에서는 이 메서드를 사용하여 관리자 권한이 없는 객체에서 속성을 제거하지 못하도록 하는 디스크립터를 만들 것이다. 특히 디스크립터가 클라이언트 객체의 소성 값을 어떻게 다루고 있는지 유심히 살펴보자.

In [53]:
class ProtectedAttribute:
    def __init__(self, requires_role=None) -> None:
        self.permission_required = requires_role
        self._name = None
    
    def __set_name__(self, owner, name):
        self._name = name
    
    def __set__(self, user, value):
        if value is None:
            raise ValueError(f"{self._name}를 None으로 설정할 수 없음")
        user.__dict__[self._name] = value
    
    def __delete__(self, user):
        if self.permission_required in user.permissions:
            user.__dict__[self._name] = None
        else:
            raise ValueError(
                f"{user!s} 사용자는 {self.permission_required} 권한이 없음"
            )
class User:
    """admin  권한을 가진 사용자만 이메일 주소를 삭제할 수 있음."""
    
    email = ProtectedAttribute(requires_role="admin")
    
    def __init__(self, username: str, email: str, permission_list: list = None) -> None:
        self.username = username
        self.email = email
        self.permissions = permission_list or []
    
    def __str__(self):
        return self.username

객체가 어떻게 동작하는지 살펴보기 전에 몇 가지 미스크립터의 기준을 확인하는 것이 중요하다. User클래스는 username과 email을 필수로 받는다. email 속성을 지워버리면 불완전한 객체가 되고 User 클래스에서 정의한 인터페이스와 맞지 않는 유효하지 않은 상태가 된다. 문제를 예방하기 위해선 이 같은 세부 사항을 확인하는 것이 중요하다. User를 사용하고자 하는 객체는 email 속성이 있는 것으로 기대하고 있다. 

이런 이유 때문에 email의 "삭제"는 단순히 None으로 설정하는 것으로 한다. admin 권한을 가진 사용자만 email 주소를 제가할 수 있다고 가정하면 다음과 같이 동작하는 것을 볼 수 있다.

In [54]:
admin = User('root', "root@d.com", ["admin"])
user = User("user", "user1@d.com", ["email", "helpdesk"])

In [55]:
admin.email

'root@d.com'

In [56]:
del admin.email

In [57]:
admin.email is None

True

In [58]:
user.email

'user1@d.com'

In [59]:
user.email = None

ValueError: email를 None으로 설정할 수 없음

In [60]:
del user.email

ValueError: user 사용자는 admin 권한이 없음

일반적으로 \__delete__ 메서드는 앞의 두 메서드에 비해 자주 사용되지는 않지만 이해의 완결성을 위해 살펴보는 것이 좋다. 

### \__set_name__(self, owner, name)

일반적으로 클래스에 디스크립터 객체를 만들 때는 디스크립터가 처리하려는 속성의 이름을 알아야 한다.

속성의 이름은 \__dict__에서 \__get__과 \__set__ 메서드로 읽고 쓸 때 사용된다.

파이썬 3.6 이전에는 디스크립터가 이 이름을 자동으로 설정하지 못했기 때문에 보통은 객체 초기화 시 명시적으로 이름을 전다랬다. 이렇게 해도 잘 동작하지만 새로운 속성에 대한 디스크립터를 추가할 때마다 이름을 복사해야 하는 불편함이 있었다.

다음은 \__set_name__이 없을 때의 전형적인 디스크립터 코드이다.

In [73]:
class DescriptorWithName:
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, value):
        if instance is None:
            return self
        logger.info("%r에서 %r 속성 가져오기" %(instance, self.name))
        print("%r에서 %r 속성 가져오기" %(instance, self.name))
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value

class ClientClass:
    descriptor = DescriptorWithName("descriptor")

In [74]:
client = ClientClass()

In [75]:
client.descriptor = "value"

In [76]:
client.descriptor

<__main__.ClientClass object at 0x06C0B830>에서 'descriptor' 속성 가져오기


'value'

속성의 이름을 두 번 쓰지 않으려면, 다음과 같이 몇 가지 트릭을 사용해야 한다. 클래스 데코레이터를 사용하거나 심지어는 메타클래스를 사용해야 한다. 

파이썬 3.6에서 새로운 메서드 \__set_name__이 추가되었는데 이 메서드는 파라미터로 디스크립터를 소유한 클래스와 디스크립터의 이름을 받는다. 디스크립터에 이 메서드를 추가하여 필요한 이름을 지정하면 된다.

일반적으로 하위 호환을 위해 \__init__ 메서드에 기본 값을 지정하고 \__set_name__을 함께 사용하는 것이 좋다.

이 방법을 사용하면 다음과 같이 다시 작성할 수 있다.

In [154]:
class DescriptorWithName:
    def __init__(self, name=None):
        self.name = name
    
    def __set_name__(self, owner, name):
        self.name = name
        
class ClientClass:
    descriptor = DescriptorWithName()

In [155]:
client = ClientClass()

In [156]:
client.descriptor = "value"

In [157]:
client.descriptor

'value'

이제 번거롭게 변수 이름을 복사하여 파라미터에 전달하지 않아도 된다.

## 디스크립터의 유형

방금 살펴본 메서드를 사용하는 디스크립터의 작동방식에 따라 디스크립터를 구분할 수 있다. 이 구분을 이해하는 것은 디스크립터를 효과적으로 사용하는데 중요한 역할을 하고, 런타임 시 경고 또는 일반적인 오류를 피하는데 도움이 된다.

디스크립터가 \__set__이나 \__delete__ 메서드를 구현했다면 데이터 디스크립터라고 부른다. 그렇지 않고 \__get__만을 구현한 디스크립터를 비데이터 디스크립터라고 부른다. \__set_name__은 이 분류에 전형 영향을 미치지 않는다.

객체의 속성을 결정할 때 데이터 디스크립터가 객체의 사전보다 우선적으로 적용되지만 비데이터 디스크립터는 그렇지 않다. 즉 비데이터 디스크립터는 객체의 사전에 디스크립터와 동일한 이름의 키가 있으면 객체의 사전 값이 적용되고 디스크립터는 절대 호출되지 않을 것이다. 반대로, 데이터 디스크립터에서는 디스크립터와 동일한 이름을 갖는 키가 사저네 존재하더라도 디스크립터 자체가 항상 먼저 호출되기 때문에 객체의 키 값은 결코 사용되지 않을 것이다.

### 비데이터 디스크립터

먼저 다음 예제를 통해 \__get__ 메서드만을 구현한 디스크립터를 살펴보자.

In [158]:
class NoneDataDescriptor:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return 42

class ClientClass:
    descriptor = NoneDataDescriptor()

평소처럼 descriptor를 호출하면 \__get__ 메서드의 결과를 얻을 수 있다.

In [159]:
client = ClientClass()
client.descriptor

42

descriptor 속성을 다른 값으로 바꾸면 이전의 값을 읽고 대신에 새로 설정한 값을 얻는다.

In [160]:
client.descriptor = 43

In [161]:
client.descriptor

43

이제 descriptor를 지우고 다시 물으면 어떤 값을 얻게 될까?

In [162]:
del client.descriptor

In [163]:
client.descriptor

42

In [164]:
del client.descriptor

AttributeError: descriptor

방금 일어난 일을 살펴보면, 처음 client 객체를 만들었을 때 descriptor 속성은 인스턴스가 아니라 클래스 안에 있다. 따라서 client 객체의 사전을 조회하면 그 값은 비어 있다.

In [92]:
vars(client)

{}

여기서 .descriptor 속성을 조회하면 client.\__dict__에서 "discriptor"라는 이름의 키를 찾지 못하고 마침내 클래스에서 디스크립터를 찾게 된다. 이것이 \__get__ 메서드의 결과가 반환되는 이유이다.

그러나 .descriptor 속성에 다른 값을 설정하면 인스턴스의 사전이 변경되므로 이제 client.\__dict__는 비어 있지 않다.

In [93]:
client.descriptor = 99
vars(client)

{'descriptor': 99}

따라서 이제 .desciptor 속성을 조회하면 객체의 \__dict__ 사전에 descriptor 키를 찾을 수 있으므로 클래스까지 검색하지 않고 바로 \__dict__ 사전에서 값을 반환한다. 때문에 디스크립터 프로토콜이 사용되지 않고 다음에 이 속성을 조회할 때는 덮어써진 99 값을 반환한다.

그 뒤에 del을 호출해 이 속성을 지우면 객체의 \__dict__ 사전에서 descriptor 키를 지운 것과 같으므로 다시 앞의 시나리오로 돌아가게 된다. 즉 다시 디스크립터 프로토콜이 다시 활성화 된다.

In [94]:
del client.descriptor
vars(client)

{}

In [95]:
client.descriptor

42

이렇게 descriptor의 속성을 설정하면 우연히 속성이 깨지게 된다. 왜냐하면 디스크립터가 \__delete__ 메서드를 구현하지 않았기 때문이다.

이런 유형의 디스크립터는 다음 세션에서 볼 예정인 \__set__ 매직멧머드를 구현하지 않았기 때문에 비데이터 디스크립터라고 한다.

In [97]:
client.__dict__

{}

In [98]:
client.descriptor = 10

In [99]:
client.__dict__

{'descriptor': 10}

In [100]:
client.descriptor = client

In [101]:
client.__dict__

{'descriptor': <__main__.ClientClass at 0x6be7750>}

In [102]:
client.desc = 12

In [103]:
client.__dict__

{'descriptor': <__main__.ClientClass at 0x6be7750>, 'desc': 12}

In [104]:
del client.__dict__

In [105]:
client.__dict__

{}

In [106]:
client.descriptor

42

In [107]:
client.descriptor = client

In [108]:
client.descriptor

In [117]:
client.asss = 12

In [119]:
class TestClass:
    def __init__(self, data):
        self.data = data

In [120]:
testclass = TestClass('123')

In [121]:
testclass.data

'123'

In [122]:
testclass.good

AttributeError: 'TestClass' object has no attribute 'good'

In [123]:
testclass.good = 12

In [124]:
testclass.good

12

In [125]:
testclass.__dict__

{'data': '123', 'good': 12}

In [126]:
TestClass.asd = 123

In [127]:
vars(TestClass)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.TestClass.__init__(self, data)>,
              '__dict__': <attribute '__dict__' of 'TestClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'TestClass' objects>,
              '__doc__': None,
              'asd': 123})

In [128]:
test = TestClass('data')

In [129]:
test.__dict__

{'data': 'data'}

In [130]:
test.asd

123

In [131]:
del test.asd

AttributeError: asd

In [132]:
del TestClass.asd

In [133]:
test.__dict__

{'data': 'data'}

In [134]:
TestClass.__dict__

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.TestClass.__init__(self, data)>,
              '__dict__': <attribute '__dict__' of 'TestClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'TestClass' objects>,
              '__doc__': None})

### 데이터 디스크립터

이제 데이터 디스크립터와의 차이를 살펴보자. 이를 위해 \__set__ 메서드를 구현한 또 다른 간단한 디스크립터를 생성할 것이다.

In [165]:
class DataDescriptor:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return 42
    def __set__(self, instance, value):
        print("%s.descriptor를 %s 값으로 설정" %(instance, value))
        instance.__dict__["descriptor"] = value

In [166]:
class ClientClass:
    descriptor = DataDescriptor()

descriptor의 반환 값을 확인해보자.

In [167]:
client = ClientClass()

In [168]:
client.descriptor

42

In [169]:
client.descriptor = 99

<__main__.ClientClass object at 0x06C0B0B0>.descriptor를 99 값으로 설정


In [170]:
client.descriptor

42

descriptor의 반환 값이 변경되지 않았다. 그러나 다른 값으로 할당하면 앞의 예와 마찬가지로 객체의 \__dict__ 사전에는 업데이트 돼야 한다.

In [171]:
vars(client)

{'descriptor': 99}

In [172]:
client.__dict__["descriptor"]

99

이렇게 되는 이유는 사실 \__set__() 메서드가 호출되면 객체의 사전에 값을 설정하기 때문이다. 그리고 데이터 디스크립터에서 속성을 조회하면 객체의 \__dict__에서 조회하는 대신 클래스의 descriptor를 먼저 조회한다. 데이터 디스크립터는 인스턴스의 \__dict__를 오버라이드하여 인스턴스 사전보다 높은 우선순위를 가지지만, 비데이터 디스크립터는 인스턴스 사전보다 낮은 우선순위를 가진다.

속성 삭제는 더 이상 동작하지 않는다.

In [173]:
del client.descriptor

AttributeError: __delete__

삭제가 되지 않는 이유는 del을 호출하면 이제 인스턴스의 \__dict__에서 속성을 지우려고 시도하는 것이 아니라 descriptor에서 \__delete__() 메서드를 호출하게 되는데 이 예제에서는 \__delete__ 메서드를 구현하지 않았기 때문이다.

이것이 데이터 디스크립터와 비데이터 디스크립터의 차이이다. 만약 디스크립터가 \__set__() 메서드를 구현했다면 객체의 사전보다 높은 우선순위를 갖는다. \__set__() 메서드를 구현하지 않았다면 객체의 사전이 우선순위를 갖고 그 다음에 디스크립터가 실행된다. 

어쩌면 발견했을지도 모르는 흥미로운 코드가 있다.

In [175]:
instance.__dict__['descriptor'] = value

이 코드에는 살펴볼 내용이 많아 보인다. 하나씩 차례로 살펴보자. 

첫째 왜 하필 "desciptor"라는 이름의 속성 값을 바꾸는 것일까? 이 예제는 단순화를 위해 디스크립터의 이름을 따로 설정하지 않았기 때문이다. 

실제로는 \__init__메서드에서 디스크립터의 이름을 받아서 내부에 저장하거나 또는 \__set_name__ 메서드를 사용해 이름을 설정할 수 있다.

인스턴스의 \__dict__ 속성에 직접 접근하는 이유는 무엇일까? 적어도 두 가지의 설명이 있을 수 있다. 첫쨰로 왜 단순하게 다름처럼 하지 않았을까 하는 것이다.

In [178]:
setattr(instance, "descriptor", value)

디스크립터의 속성에 무언가 할당하려고 하면 \__set__ 메서드가 호출된다는 것을 기억하자. 따라서 setattr()을 사용하면 디스크립터의 \__set__ 메서드가 호출되고, \__set__메서드는 setattr을 호줄하고 다시 호출되는 무한루프가 발생한다.

instance.dscriptor = value와 같은 할당 표현식도 같은 이유로 무한루프를 유발한다.

그런데 디스크립터가 모든 인스턴스의 프로퍼티 값을 보관할 수 없는 이유는 뭘까?

클라이언트 클래스는 이미 디스크립터의 참조를 가지고 있다. 디스크립터가 다시 클라이언트 객체를 참조하면 순환 종속성이 생기게 되어 가비지 커렉션이 되지 않는 문제가 생긴다. 서로를 가리키고 있기 때문에 참조 카운트가 제거 임계치 이하로 떨어지지 않는다.

이에 대한 대안은 weakref 모듈에 있는 약한 참조를 사용하여 약한 참조 키 사전을 만드는 것이다. 약한 참조에 대한 내용은 잠시 후에 알아보자. 이러한 구현 방식을 따르는 것이 상당히 일반적이어서 이 책에서도 같은 방법을 사용한다.

## 디스크립터 실전

이제 디스크립터가 무엇인지 어떻게 작동하는지 그리고 그 뒹 있는 주요 개념이 무엇인지 살펴보았다. 이제 디스크립터를 통해 처리할 수 있는 몇 가지 상황에 대해 살펴본다.

### 디스크립터를 사용한 애플리케이션

